In [1]:
%%capture
# Install the latest Tensorflow version.
!pip install tensorflow_text
!pip install bokeh
!pip install simpleneighbors[annoy]
!pip install tqdm

In [2]:
import bokeh
import bokeh.models
import bokeh.plotting
import numpy as np
import os
import pandas as pd
import tensorflow.compat.v2 as tf
import tensorflow_hub as hub
from tensorflow_text import SentencepieceTokenizer
import sklearn.metrics.pairwise
from tqdm import tqdm
from tqdm import trange
import re
import string
from tensorflow.keras import Model
from tensorflow.keras.layers import Dot, Embedding, Flatten
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE
from sklearn.feature_extraction.text import CountVectorizer ,TfidfVectorizer

In [ ]:

import zipfile
from google.colab import drive

drive.mount('/content/drive/')
#nous allons extraire nos données qui se trouvent dans un fichier zip dans notre drive 
zip_ref = zipfile.ZipFile("/content/drive/MyDrive/rebuilt.Cross-Language-Dataset-master.zip", 'r')
zip_ref.extractall("/tmp")
zip_ref.close()

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


error: ignored

In [4]:
#nous allons experimenter une partie de nos donnée dans cette partie on commence par le corpus APR 
English_Dataset = tf.data.Dataset.list_files("/tmp/Cross-Language-Dataset-master/dataset/chunks/APR/en/*.txt")
Frensh_Dataset = tf.data.Dataset.list_files("/tmp/Cross-Language-Dataset-master/dataset/chunks/APR/fr/*.txt")


In [5]:
#nous allons organiser nos fichier text de sort a ce que les fichier en francais aille leur traduction anglaise
English_DatasetFiles=list(English_Dataset.as_numpy_iterator())
English_DatasetFiles.sort()
Frensh_DatasetFiles=list(Frensh_Dataset .as_numpy_iterator())
Frensh_DatasetFiles.sort()

In [6]:
#cette fonction permet d'extraire les donnees des fichier
def load_data(Files):
  sentences=[]
  for file_content in Files: 
    with open(file_content, "r") as f:
      data = f.read()
      sentences.append(data)
  return sentences

english_sentences=load_data(English_DatasetFiles)
french_sentences=load_data(Frensh_DatasetFiles)


In [7]:
for i in range(5):
  print('Sample :',i)
  print(english_sentences[i])
  print(french_sentences[i])
  print('-'*50)

Sample : 0
the association gregory lemarchal

l' association gregory lemarchal

--------------------------------------------------
Sample : 1
after the dresden file

après les dossier dresden

--------------------------------------------------
Sample : 2
the tradition of lanfeust troy
invention of reverse auctions

la tradition lanfeust de troy
invention des ventes aux enchères

--------------------------------------------------
Sample : 3
the manner of dos passos

la manière de dos passos

--------------------------------------------------
Sample : 4
trace of fat in this book

trace de gras dans le livre

--------------------------------------------------


In [8]:
from collections import defaultdict

In [12]:
def Word_Embedding(sentences,contexts):
   for sentence in sentences:
     words=sentence.split()
     for i,word in enumerate(words):
       condidate=[i+j for j in range (-1,2)if j!=0]
       actuel=[c for c in condidate if 0<= c<len(words)]
       contexts[word]= contexts[word]+[words[a] for a in actuel]
   

In [ ]:
def Vectorisation(contexts,sentences):
   vectorizer=TfidfVectorizer()
   vectorizer.fit(sentences)
   vectors={}
   for k,v in dict(contexts).items():
      str=' '.join(v)
      vectors[k]=np.around(vectorizer.transform([str]).toarray(),2)[0]
   return vectors

In [13]:
contexts=defaultdict(list)
Word_Embedding(english_sentences,contexts)
for k,v in contexts.items():
  print(k)
  print(v)
  print()


Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
['driver', 'pulo']

pulo
['titus', 'in']

masterpiece
['the', 'rome']

killing
['of']

transitional
['film']

similar
['costner', 'to']

designs
['to', 'by']

bradstreet
['tim']

solo
['of', 'hours', '2nd', 'album', 'album', 'a', 'career', 'early', 'career', 'with', 'piano', 'biondi', 'in', 'albums', 'first', 'album', 'true', 'album', 'mercury', 'great', 'huge', 'guitar', 'sax', 'kilos', 'desmond', 'pseudo', 'a', 'opus', 'keith', 'albums']

mel
['of', 'brooks']

brooks
['mel']

ballet
['of']

spanish
['the', 'inquisition', 'this', 'version']

inquisition
['spanish']

revolution
['the', 'date']

date
['revolution', 'of', 'the', 'of', 'to', 'structures']

guerilla
['the', 'in']

bolivia
['in']

hot
['of', 'shots', 'red', 'refrain', 'red', 'chili', 'of', 'tuna']

mulholland
['of', 'drive']

drive
['mulholland']

drebin
['frank', 'in']

delirious
['of', 'smart']

smart
['delirious', 'offers']

offers
['smart', 'a

In [24]:
vectors=Vectorisation(context,english_sentences)
#print(vectorizer.get_feature_names())
print()
for k,v in vectors.items():
  print(k)
  print(v)
  print()

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
[0. 0. 0. ... 0. 0. 0.]

pulo
[0. 0. 0. ... 0. 0. 0.]

masterpiece
[0. 0. 0. ... 0. 0. 0.]

killing
[0. 0. 0. ... 0. 0. 0.]

transitional
[0. 0. 0. ... 0. 0. 0.]

similar
[0. 0. 0. ... 0. 0. 0.]

designs
[0. 0. 0. ... 0. 0. 0.]

bradstreet
[0. 0. 0. ... 0. 0. 0.]

solo
[0. 0. 0. ... 0. 0. 0.]

mel
[0. 0. 0. ... 0. 0. 0.]

brooks
[0. 0. 0. ... 0. 0. 0.]

ballet
[0. 0. 0. ... 0. 0. 0.]

spanish
[0. 0. 0. ... 0. 0. 0.]

inquisition
[0. 0. 0. ... 0. 0. 0.]

revolution
[0. 0. 0. ... 0. 0. 0.]

date
[0. 0. 0. ... 0. 0. 0.]

guerilla
[0. 0. 0. ... 0. 0. 0.]

bolivia
[0. 0. 0. ... 0. 0. 0.]

hot
[0. 0. 0. ... 0. 0. 0.]

mulholland
[0. 0. 0. ... 0. 0. 0.]

drive
[0. 0. 0. ... 0. 0. 0.]

drebin
[0. 0. 0. ... 0. 0. 0.]

delirious
[0. 0. 0. ... 0. 0. 0.]

smart
[0. 0. 0. ... 0. 0. 0.]

offers
[0. 0. 0. ... 0. 0. 0.]

succession
[0. 0. 0. ... 0. 0. 0.]

steve
[0. 0. 0. ... 0. 0. 0.]

carell
[0. 0. 0. ... 0. 0. 0.]

lone
[

In [21]:
context2=defaultdict(list)
Word_Embedding(french_sentences,context2)
vectorsF=Vectorisation(context2,french_sentences)

In [22]:
for k,v in vectorsF.items():
  print(k)
  print(v)
  print()

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
[0. 0. 0. ... 0. 0. 0.]

kecharitomene
[0. 0. 0. ... 0. 0. 0.]

marco
[0. 0. 0. ... 0. 0. 0.]

polo
[0. 0. 0. ... 0. 0. 0.]

live
[0. 0. 0. ... 0. 0. 0.]

avalanche
[0. 0. 0. ... 0. 0. 0.]

hits
[0. 0. 0. ... 0. 0. 0.]

planétaires
[0. 0. 0. ... 0. 0. 0.]

clip
[0. 0. 0. ... 0. 0. 0.]

vidéo
[0. 0. 0. ... 0. 0. 0.]

gad
[0. 0. 0. ... 0. 0. 0.]

elmaleh
[0. 0. 0. ... 0. 0. 0.]

russel
[0. 0. 0. ... 0. 0. 0.]

crowe
[0. 0. 0. ... 0. 0. 0.]

sandler
[0. 0. 0. ... 0. 0. 0.]

nullissime
[0. 0. 0. ... 0. 0. 0.]

jury
[0. 0. 0. ... 0. 0. 0.]

soi-disant
[0. 0. 0. ... 0. 0. 0.]

grotesque
[0. 0. 0. ... 0. 0. 0.]

conscience
[0. 0. 0. ... 0. 0. 0.]

judd
[0. 0. 0. ... 0. 0. 0.]

rosenthal
[0. 0. 0. ... 0. 0. 0.]

sensuelle
[0. 0. 0. ... 0. 0. 0.]

kathy
[0. 0. 0. ... 0. 0. 0.]

bates
[0. 0. 0. ... 0. 0. 0.]

scenes
[0. 0. 0. ... 0. 0. 0.]

habillements
[0. 0. 0. ... 0. 0. 0.]

fiml
[0. 0. 0. ... 0. 0. 0.]

apréhention